In [10]:
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore

In [12]:
def read_csv_column(csv_path, column_name):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    return df[column_name].tolist()

def read_csv_with_weight(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8', low_memory=False)
    # 转发数,评论数,点赞数
    # comments = df['评论数'].tolist()
    # reposts = df['转发数'].tolist()
    # likes = df['点赞数'].tolist()
    weibos = df['微博正文'].tolist()

    new_list = []
    for weibo in weibos:
        weibo = weibo.split()
        new_list.append(weibo)

    return new_list

In [16]:
for i in range(10):
    if i < 9:
        index = '0' + str(i + 1)
    else:
        index = str(i + 1)
    trump_content = read_csv_with_weight('../dataset/trump/trump' + index + '.csv')

    dictionary = Dictionary(trump_content)
    bow_corpus = [dictionary.doc2bow(doc) for doc in trump_content]
    lda_model_tfidf = LdaMulticore(bow_corpus, num_topics=30, id2word=dictionary, passes=10, workers=4)
    item = []
    for j in range(len(trump_content)):
        item.extend(trump_content[j])

    bow_vector = dictionary.doc2bow(item)

    print(index + '月')
    for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
        print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
    print()

01月
Score: 0.09480325132608414	 Topic: 0.031*"协议" + 0.019*"经济" + 0.019*"特朗普" + 0.019*"贸易" + 0.018*"美国" + 0.017*"第一" + 0.017*"阶段" + 0.017*"中国" + 0.016*"市场" + 0.015*"签署"
Score: 0.09194488078355789	 Topic: 0.057*"特朗普" + 0.049*"伊朗" + 0.048*"伊拉克" + 0.038*"美国" + 0.024*"美军" + 0.015*"袭击" + 0.012*"空袭" + 0.010*"战争" + 0.009*"巴格达" + 0.008*"总统"
Score: 0.0775013193488121	 Topic: 0.079*"伊朗" + 0.053*"特朗普" + 0.043*"美国" + 0.015*"制裁" + 0.012*"协议" + 0.011*"中东" + 0.010*"战争" + 0.007*"伊核" + 0.006*"发表" + 0.006*"军事"
Score: 0.07397221773862839	 Topic: 0.095*"伊朗" + 0.054*"特朗普" + 0.048*"美国" + 0.028*"苏莱曼尼" + 0.021*"美军" + 0.021*"袭击" + 0.016*"导弹" + 0.016*"暗杀" + 0.011*"基地" + 0.011*"客机"
Score: 0.06110473349690437	 Topic: 0.062*"特朗普" + 0.044*"弹劾" + 0.041*"美国" + 0.035*"参议院" + 0.024*"弹劾案" + 0.019*"总统" + 0.018*"审理" + 0.018*"正式" + 0.016*"众议院" + 0.014*"边境"
Score: 0.04778200015425682	 Topic: 0.053*"特朗普" + 0.045*"美国" + 0.023*"总统" + 0.011*"国会" + 0.010*"达沃斯" + 0.010*"关系" + 0.009*"政府" + 0.008*"伊朗" + 0.007*"经济" + 0.007*"众议院"
Scor